In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

In [ ]:
root = 'C:\\Users\\Michał\\Desktop\\nkjp+wiki-lemmas-all-100-cbow-ns.txt\\nkjp+wiki-lemmas-all-100-cbow-ns.txt'

from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open(root, encoding="utf8",errors='ignore')

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [ ]:
grades = []
posReviews =  open(r'C:\Users\Michał\Desktop\iet\2_semestr\PJN\2\LowerWithoutPunctuationWithoutHapax\allPosReviewsLowerWithoutPunctuationWithoutHapax.txt', 'r',errors='ignore',encoding="utf8")

posLines = posReviews.readlines()

negReviews =open(r'C:\Users\Michał\Desktop\iet\2_semestr\PJN\2\LowerWithoutPunctuationWithoutHapax\allNegReviewsLowerWithoutPunctuationWithoutHapax.txt', 'r',errors='ignore',encoding="utf8")

negLines = negReviews.readlines()

for i in posLines:
    grades.append(1)
for i in negLines:
    grades.append(0)

In [ ]:
wholeData = posLines + negLines
print(len(grades))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(wholeData, grades, test_size=0.20, random_state=42)

In [ ]:
tokenizer = Tokenizer(num_words=25000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
model = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())

In [ ]:
history = model.fit(X_train, y_train, batch_size=128, epochs=40, verbose=1, validation_split=0.2)

In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])